<a href="https://colab.research.google.com/github/lowfuel/DiscoDiffusion-Warp-gobig/blob/lowfuel/CLIP_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIP Evaluator

Please considering supporting me [Patreon](https://www.patreon.com/user?u=255893&fan_landing=true) to keep this notebook updated and improving. Thanks!

This notebook allows you to provide some sample pieces of art, then have CLIP evaluate the images and tell you who it thinks the artist is.

Please be sure to add the actual artist to the list if they aren't already there, and message me on Discord (lowfuel) if you have an artist I should add to the list.

## How to interpret the results

- If CLIP agrees that your artist created the images, you can be confident in using that artist's name in your prompts.
- If CLIP disagrees, it means that some other artist (in our list) is generating a response that is more similar to your source art.
-- This doesn't mean your artist is not well understood by CLIP, but it may mean that another artist (or artists) have a better representation in the models compared to yours.
- If CLIP disagrees, and the artist it thinks made it is also a poor match, it could mean that there was no stong artistic recognition, but perhaps some other element of the image triggered a response.

### Credits & Changelog ⬇️

#### Credits

by Jason Hough (lowfuel)

#### License

Licensed under the MIT License

Copyright (c) 2022 Jason Hough 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# 1. Set Up

In [ ]:
#@title 1.1 Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_ecc_note)

In [ ]:
#@title 1.2 Prepare Folders
import subprocess, os, sys, ipykernel

def gitclone(url):
  res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipi(modulestr):
  res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def pipie(modulestr):
  res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def wget(url, outputdir):
  res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

import os
root_path = os.getcwd()

wget("https://github.com/lowfuel/DiscoDiffusion-Warp-gobig/blob/main/prompts.txt", root_path)

def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)


In [ ]:
#@title ### 1.3 Install and import dependencies

import pathlib, shutil, os, sys

if not is_colab:
  # If running locally, there's a good chance your env will need this in order to not crash upon np.matmul() or similar operations.
  os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

PROJECT_DIR = os.path.abspath(os.getcwd())

multipip_res = subprocess.run(['pip', 'install', 'lpips', 'datetime', 'timm', 'ftfy', 'einops', 'pytorch-lightning', 'omegaconf'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(multipip_res)

import os
from os import path
import sys
from typing import final

from attr import has
from numpy import average
import torch
from torch import nn

try:
  from CLIP import clip
except:
  if not os.path.exists("CLIP"):
    gitclone("https://github.com/openai/CLIP")
  sys.path.append(f'{PROJECT_DIR}/CLIP')
import gc
from statistics import mean
from PIL import Image, ImageOps
import urllib.request, urllib.error, urllib.parse

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
device = DEVICE # At least one of the modules expects this name..

if torch.cuda.get_device_capability(DEVICE) == (8,0): ## A100 fix thanks to Emad
  print('Disabling CUDNN for A100 gpu', file=sys.stderr)
  torch.backends.cudnn.enabled = False

# 2. CLIP model settings

In [ ]:
ViTB32 = True #@param{type:"boolean"}
ViTB16 = True #@param{type:"boolean"}
ViTL14 = True #@param{type:"boolean"}
ViTL14_336 = True #@param{type:"boolean"}
RN101 = True #@param{type:"boolean"}
RN50 = True #@param{type:"boolean"}
RN50x4 = True #@param{type:"boolean"}
RN50x16 = True #@param{type:"boolean"}
RN50x64 = True #@param{type:"boolean"}

modellist = []
if RN50x64 == True: modellist.append('RN50x64')
if ViTB16 == True: modellist.append('ViT-B/16')
if ViTL14 == True: modellist.append('ViT-L/14')
if ViTL14_336 == True: modellist.append('ViT-L/14@336px')
if RN50 == True: modellist.append('RN50')
if RN50x4 == True: modellist.append('RN50x4')
if RN50x16 == True: modellist.append('RN50x16')
if ViTB32 == True: modellist.append('ViT-B/32')
if RN101 == True: modellist.append('RN101')

def load_clip_model(model_name):
    model, preprocess = clip.load(model_name, jit=False, device=device)
    return model, preprocess

def clipit_text(prompt, model):
    text = model.encode_text(clip.tokenize(prompt).to(device)).float()
    return text

def clipit_image(im_prompt, model):
    image = model.encode_image(im_prompt.to(device)).float()
    return image

def evalprompt_cos(t_prompt, t_comp):
    #Get similarity between two tensors
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    t_con = cos(t_prompt, t_comp)
    similarity = t_con.item()
    return similarity

def Average(lst):
    return sum(lst) / len(lst)

def loadprompts(textfile):
    prompts = []
    with open(textfile, encoding="utf-8") as f:
        for line in f:
            prompts.append(line.strip())
    return(prompts)
    
def scoreprompts(evalprompt, images, prompts, modelname):
    model, preprocess = load_clip_model(modelname)
    scores = []
    if evalprompt not in prompts:
        prompts.append(evalprompt)
    for prompt in prompts:
        if len(images) > 0:
            t_imgs = []
            for image in images:
                p_image = preprocess(image.unsqueeze(0).to(device))
                t_img = clipit_image(p_image, model)
                t_imgs.append(t_img) # our image tensors for this check
            most_similar_prompt = ""
            high_score = 0.0
            prompt_scores = []
            for checkprompt in prompts:
                t_prompt = clipit_text(checkprompt, model)
                for t_img in t_imgs:
                    score = evalprompt_cos(t_prompt, t_img)
                    if score > high_score:
                        most_similar_prompt = (f'"{checkprompt}" at {score:.{3}}')
                        high_score = score
                    if checkprompt == prompt:
                        prompt_scores.append(score)
            final_score = Average(prompt_scores)
            print(f'"{prompt}" scored {final_score:.{3}}, CLIP thought it was {most_similar_prompt}.')
        else:
            final_score = 0.0
        scores.append(final_score)
    return scores


# 3. Enter prompt and select sample images

In [ ]:
evalprompt = "by Antony Carlyon" #@param{type:"string"}

image1 = "" #@param{type:"string"}
image2 = "" #@param{type:"string"}
image3 = "" #@param{type:"string"}
image4 = "" #@param{type:"string"}

# 4. RESULTS

In [ ]:
images = []
i1 = urllib.request.urlopen(image1).read()
i2 = urllib.request.urlopen(image2).read()
i3 = urllib.request.urlopen(image3).read()
i4 = urllib.request.urlopen(image4).read()
images.append(i1)
images.append(i2)
images.append(i3)
images.append(i4)

prompts = loadprompts("prompts.txt")

image_results = [] 
for image in images:
    results = [] # tuple with evalprompt_score, best_scores, modelname
    for modelname in modellist:
        result = scoreprompts(evalprompt, image, prompts, modelname)
        results.append(result)
    image_results.append(results)

print(image_results)